# Demo ConvNet para identificar LETRAS & NÚMEROS (realiza sólo validación del modelo ya entrenada y grabado en Drive)
Se usa el dataset de imágenes obtenido de http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

1) Cargar librerías:

In [0]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from IPython.display import Image

import os
import os.path

import json
from keras.models import model_from_json
from keras.models import load_model

import numpy as np 
import matplotlib.pyplot as plt

print ("Librerías cargadas.")

2) Monta el Drive para poder acceder a los archivos:

In [0]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demoConvNet-Letras'

# define los nombres de los archivos a utilizar para leer/grabar el modelo
history_file_name = path + '/Model/CNN_L_history_dump_final.json'
weights_file_name = path + '/Model/CNN_L_model_final.h5'
model_json_file_name = path + '/Model/CNN_L_model_final.json'

3) Carga el modelo ya entrenado:




In [0]:
# carga modelo ya grabado de ConvNet
if os.path.isfile(model_json_file_name):
    classifier = load_model(weights_file_name)

    if os.path.isfile(history_file_name):
      h = json.load(open(history_file_name, 'r'))
      print("Modelo cargado: [", weights_file_name, "], [", history_file_name, "] y [", model_json_file_name, "] ")
    else: 
      print("No se encuentra modelo para cargar")
else:   
    print("No se encuentra modelo para cargar")

# muestra el modelo cargado
print(classifier.summary())

# carga la lista de clases si no está definida   
import csv
with open( path + '/Model/clasesLetrasNros.csv', mode='r') as csvfile:
    all_classes = list(csv.reader(csvfile))[0]
print('Definición de las clases: ', all_classes)


---

4) Muestra estadísticas y resultados del probar el modelo:

In [0]:
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# define función auxiliar para mostrar resultado de cada imágen
def testImage(file_name, image_sample, classDesired, showPredictOK):
    result = classifier.predict(image_sample)
    
    # identifica mejor
    bestPos = np.argmax(result, axis=1)
    clasPred = str(all_classes[int(bestPos)])

    prediction = clasPred + "" + str(result[0][bestPos]) + "" 
    if clasPred == classDesired:
      res = True      
    else:
      res = False
      prediction = prediction + "!"

    # muestra resultados (solo con error)
    if ((not res) or showPredictOK):
      print("> ", file_name,": " , prediction)
      img = Image(file_name, width = "80", height = "50")
      display(img)

    return res, clasPred

# define función auxiliar para mostrar resultado de cada directorio
def testAllClass(classDesired):
  cantOK = 0
  cantNOK = 0
  predict_path = "".join([path, '/Letras/test/', str(classDesired)])
  print("\n")
  y_classReal = []
  y_classRes = []
  for file in os.listdir(predict_path):
      if not file.startswith('.'):
          file = predict_path + "/" + file

          image_sample = image.load_img(file, target_size = (128, 128))
          image_sample = image.img_to_array(image_sample)
          image_sample = np.expand_dims(image_sample, axis = 0)
          
          result, clRes = testImage(file, image_sample, classDesired, False)
          if (result):
            cantOK = cantOK + 1
          else:
            cantNOK = cantNOK + 1

          y_classReal.append(classDesired)
          y_classRes.append(clRes)

  print("\nTOTAL CLASS", classDesired,": ", cantOK+cantNOK, ": Detectado OK ", cantOK, "imágenes - Detectado con Error ", cantNOK, "imágenes.")  
  print('con una Exactitud de %f' % accuracy_score(y_classReal, y_classRes))

  return cantOK, cantNOK, y_classReal, y_classRes

# procesa las imágenes de la carpeta <Test>
y_tests = []
y_preds = []
okGral = 0 
NokGral = 0
all_dirs = os.listdir("".join([path, '/Letras/test']))
for each_dir in all_dirs:
  print("\n--- Procesando ", each_dir)
  ok, nok, tests, preds = testAllClass(each_dir)
  print("\n--------------------------------------------------------------------------------------------------------------- ")  
  okGral = ok + okGral 
  NokGral = nok + NokGral
  y_tests.extend(tests)
  y_preds.extend(preds)


print("\n===========================================================================================================================")
print("\n= TOTAL GENERAL ", okGral+NokGral, ": Detectado OK ", okGral, "imágenes - Detectado con Error ", NokGral, "imágenes.\n\n")

 
print('\n= Exactitud: %f' % accuracy_score(y_tests, y_preds))
  
print('\n= Matriz de Confusión: ')
print(confusion_matrix(y_tests, y_preds, all_classes))

print("\n= Reporte de Clasificación: ")
print(classification_report(y_tests, y_preds))

print("\n===========================================================================================================================")
